### import all libraries

In [115]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geopy.geocoders import Nominatim 
import numpy as np
import time
import folium
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

### Scrap data from wikipedia using Beautifulsoup

In [8]:
url='https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India'
country = 'India',
city = 'Hyderabad'
geolocator = Nominatim()
page = requests.get(url)
parsedHTML = BeautifulSoup(page.text, 'html.parser')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


In [9]:
citiesContainers = parsedHTML.find_all('div', class_='mw-category-group')
cities = [];
c = 1;
for citiesContainer in citiesContainers:
    for li in citiesContainer.find_all('li'):
        cities.append(li.text.strip().split(',')[0])
print(len(cities))

200


### collected all cities list

In [10]:
cities_df = pd.DataFrame({'Neighbourhoods': cities})
cities_df.head(10)

,Neighbourhoods
0,A. S. Rao Nagar
1,A.C. Guards
2,Abhyudaya Nagar
3,Abids
4,Adikmet
5,Afzal Gunj
6,Aghapura
7,Aliabad
8,Alijah Kotla
9,Allwyn Colony


### collect all geo coordiantes for each neighnourhood

In [ ]:
latitudes = []
longitudes = []
errorCities = []
for neighbourhood in cities:
    print(neighbourhood)
    address = "{},{},{}".format(neighbourhood, city, country)
    retries = 3
    while (retries > 0):
        try:
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
            latitudes.append(latitude)
            longitudes.append(longitude)
            retries = 0
            if neighbourhood in errorCities:
                errorCities.remove(neighbourhood)
        except Exception as e:
            print("ERROR: ", address, e)
            if not neighbourhood in errorCities:
                errorCities.append(neighbourhood)
            retries = retries - 1
            time.sleep(1)

In [ ]:
cities_with_geo_coordinates = filter(lambda city: not city in errorCities, cities)
cities_with_geo_coordinates = [city for city in cities_with_geo_coordinates]

In [24]:
cities_with_geo_coordinates[0:10]

['A. S. Rao Nagar',
 'A.C. Guards',
 'Abhyudaya Nagar',
 'Abids',
 'Adikmet',
 'Aghapura',
 'Aliabad',
 'Alijah Kotla',
 'Allwyn Colony',
 'Alwal']

### prepare a dataframe with required features

In [61]:
cities_df = pd.DataFrame({'Neighbourhood': cities_with_geo_coordinates, 'Latitude': latitudes, 'Longitude': longitudes})

In [62]:
cities_df.head(10)

,Neighbourhood,Latitude,Longitude
0,A. S. Rao Nagar,17.479950,78.556834
1,A.C. Guards,17.402804,78.459487
2,Abhyudaya Nagar,17.337661,78.564716
3,Abids,17.389478,78.477182
4,Adikmet,17.409550,78.513094
5,Aghapura,17.389178,78.465273
6,Aliabad,17.345630,78.472680
7,Alijah Kotla,17.360545,78.480102
8,Allwyn Colony,17.504362,78.414985
9,Alwal,17.502229,78.508858


In [63]:
cities_df.tail(10)

,Neighbourhood,Latitude,Longitude
161,Secunderabad,17.469089,78.505918
162,Serilingampally,17.465657,78.340672
163,Shahran Market,17.359578,78.474090
164,Shanker Mutt,17.400764,78.507340
165,Shivam Road,17.336795,78.602895
166,Sikh Village,17.460098,78.487287
167,Silpa Avenue Colony,17.439550,78.367562
168,Sindhi Colony,17.441219,78.481048
169,Sitaphalmandi,17.429733,78.517426
170,Somajiguda,17.425505,78.458520


In [64]:
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(city, latitude, longitude))

The geograpical coordinate of Hyderabad are 17.38878595, 78.46106473453146.


### View map of all neighbourhoods

In [ ]:
hyderabad_map = folium.Map(location=[latitude, longitude], zoom_start=10)
for neighbourhood, latiude, longitude in zip(cities_df['Neighbourhood'], cities_df['Laitude'], cities_df['Longitude']):
    label = neighbourhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latiude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(hyderabad_map) 
hyderabad_map

In [65]:
LIMIT = 100
radius = 500

CLIENT_ID = 'ZPT23A2KX3YWPVWSILJD5ABOSZIM241NTZU3PWVDWTMRWVFO'
CLIENT_SECRET = 'C25N1NKBSJUWOXIAGSHR1WB4135VYYH4TK3LEOB4S2JAX0OQ'
VERSION = '20180605'

### Use folium to get venues of each neighbourhoods

In [66]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
hyderabad_venues = getNearbyVenues(names=cities_df['Neighbourhood'],
                                   latitudes=cities_df['Latitude'],
                                   longitudes=cities_df['Longitude'],
                                  )

In [68]:
print(hyderabad_venues.shape)
hyderabad_venues.head()

(1132, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,A. S. Rao Nagar,17.47995,78.556834,Cafe Coffee Day,17.481262,78.555077,Café
1,A. S. Rao Nagar,17.47995,78.556834,Max,17.478846,78.558801,Clothing Store
2,A. S. Rao Nagar,17.47995,78.556834,Foodworld,17.481645,78.554110,Department Store
3,A. S. Rao Nagar,17.47995,78.556834,Swagath Grand,17.482022,78.553261,Indian Restaurant
4,A. S. Rao Nagar,17.47995,78.556834,China Town,17.480912,78.560210,Chinese Restaurant


In [72]:
hyderabad_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A. S. Rao Nagar,6,6,6,6,6,6
A.C. Guards,12,12,12,12,12,12
Abhyudaya Nagar,5,5,5,5,5,5
Abids,17,17,17,17,17,17
Adikmet,4,4,4,4,4,4
...,...,...,...,...,...,...
Sikh Village,7,7,7,7,7,7
Silpa Avenue Colony,6,6,6,6,6,6
Sindhi Colony,14,14,14,14,14,14


In [74]:
print('There are {} uniques categories.'.format(len(hyderabad_venues['Venue Category'].unique())))

There are 163 uniques categories.


### One hot enconding for categorial values

In [92]:
# one hot encoding
hyderabad_onehot = pd.get_dummies(hyderabad_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hyderabad_onehot['Neighborhood'] = hyderabad_venues['Neighborhood'] 
hyderabad_onehot.head()

,ATM,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Tennis Court,Thai Restaurant,Theme Park,Tourist Information Center,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
hyderabad_grouped = hyderabad_onehot.groupby('Neighborhood').mean().reset_index()
print(hyderabad_grouped.shape)
hyderabad_grouped.head()

(161, 163)


,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,...,Tennis Court,Thai Restaurant,Theme Park,Tourist Information Center,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Women's Store
0,A. S. Rao Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A.C. Guards,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abhyudaya Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abids,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Adikmet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
num_top_venues = 10

for hood in hyderabad_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = hyderabad_grouped[hyderabad_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A. S. Rao Nagar----
                venue  freq
0   Indian Restaurant  0.17
1    Department Store  0.17
2      Clothing Store  0.17
3  Chinese Restaurant  0.17
4               Diner  0.17


----A.C. Guards----
                       venue  freq
0      Hyderabadi Restaurant  0.17
1          Indian Restaurant  0.17
2                      Hotel  0.08
3  Middle Eastern Restaurant  0.08
4           Department Store  0.08


----Abhyudaya Nagar----
                venue  freq
0   Indian Restaurant   0.4
1               Hotel   0.2
2        Burger Joint   0.2
3  English Restaurant   0.2
4           Multiplex   0.0


----Abids----
               venue  freq
0              Hotel  0.18
1  Indian Restaurant  0.18
2          Juice Bar  0.12
3   Department Store  0.06
4              Diner  0.06


----Adikmet----
               venue  freq
0                Gym  0.50
1               Café  0.25
2     Clothing Store  0.25
3                ATM  0.00
4  Mobile Phone Shop  0.00


----Aghapura----
     

                            venue  freq
0                   Metro Station  0.25
1                   Deli / Bodega  0.25
2                     Bus Station  0.25
3                  Farmers Market  0.25
4  Multicuisine Indian Restaurant  0.00


----Fateh Nagar----
                            venue  freq
0                   Train Station   1.0
1                             ATM   0.0
2  Multicuisine Indian Restaurant   0.0
3                   Metro Station   0.0
4              Mexican Restaurant   0.0


----Ferozguda----
                       venue  freq
0          Indian Restaurant  0.67
1                   Tea Room  0.33
2                        ATM  0.00
3         Mexican Restaurant  0.00
4  Middle Eastern Restaurant  0.00


----Film Nagar----
                     venue  freq
0                     Café  0.43
1           Sandwich Place  0.14
2  New American Restaurant  0.14
3         Greek Restaurant  0.14
4               Restaurant  0.14


----Gachibowli----
                venue  freq


                venue  freq
0          Food Court  0.17
1                Café  0.17
2      Ice Cream Shop  0.17
3  Chinese Restaurant  0.17
4                 Gym  0.17


----Lallaguda----
                            venue  freq
0                   Metro Station  0.25
1                  Cricket Ground  0.25
2                   Train Station  0.25
3                          Bakery  0.25
4  Multicuisine Indian Restaurant  0.00


----Lingampally----
            venue  freq
0        Pharmacy  0.33
1     Snack Place  0.33
2  Farmers Market  0.33
3             ATM  0.00
4   Movie Theater  0.00


----Lingojiguda----
                 venue  freq
0                 Lake  0.29
1        Movie Theater  0.29
2       Scenic Lookout  0.14
3  Indie Movie Theater  0.14
4     Asian Restaurant  0.14


----Madannapet----
                       venue  freq
0                   Pharmacy   0.5
1                       Café   0.5
2                Outlet Mall   0.0
3      Outdoors & Recreation   0.0
4  Middle East

           venue  freq
0    Comedy Club  0.25
1  Movie Theater  0.25
2          Diner  0.25
3         Bakery  0.25
4            ATM  0.00


----Raidurg----
                            venue  freq
0               Indian Restaurant   0.4
1                      Auditorium   0.2
2            Fast Food Restaurant   0.2
3                      Restaurant   0.2
4  Multicuisine Indian Restaurant   0.0


----Raj Bhavan Road----
                  venue  freq
0     Indian Restaurant  0.14
1  Fast Food Restaurant  0.11
2           Pizza Place  0.07
3           Coffee Shop  0.07
4        Sandwich Place  0.07


----Ramachandrapuram----
                            venue  freq
0                        Pharmacy  0.33
1                     Pizza Place  0.33
2                          Bakery  0.33
3  Multicuisine Indian Restaurant  0.00
4              Mexican Restaurant  0.00


----Ramanthapur----
                            venue  freq
0              Turkish Restaurant   1.0
1                            

In [129]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top 10 venues

In [147]:

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{}'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hyderabad_grouped['Neighborhood']

for ind in np.arange(hyderabad_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hyderabad_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(8)

,Neighborhood,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,A. S. Rao Nagar,Clothing Store,Chinese Restaurant,Indian Restaurant,Department Store,Café,Diner,Women's Store,Dumpling Restaurant,Fast Food Restaurant,Farmers Market
1,A.C. Guards,Hyderabadi Restaurant,Indian Restaurant,Shoe Store,South Indian Restaurant,Hotel,Department Store,Middle Eastern Restaurant,Ice Cream Shop,Bakery,Café
2,Abhyudaya Nagar,Indian Restaurant,Hotel,Burger Joint,English Restaurant,Women's Store,Dumpling Restaurant,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,Abids,Hotel,Indian Restaurant,Juice Bar,Fast Food Restaurant,Electronics Store,Shoe Store,Shopping Mall,Department Store,Diner,South Indian Restaurant
4,Adikmet,Gym,Café,Clothing Store,Donut Shop,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
5,Aghapura,Afghan Restaurant,Platform,Indian Restaurant,Bakery,Women's Store,Dumpling Restaurant,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
6,Aliabad,Gym,Chinese Restaurant,Smoke Shop,Ice Cream Shop,Snack Place,Asian Restaurant,Women's Store,Donut Shop,Fast Food Restaurant,Farmers Market
7,Allwyn Colony,ATM,Bar,Mattress Store,Dumpling Restaurant,Flea Market,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant


### Get 5 clusters

In [148]:
kclusters = 5

hyderabad_grouped_clustering = hyderabad_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hyderabad_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 0, 2, 2, 2, 2, 2, 2, 0], dtype=int32)

In [149]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hyderabad_merged = cities_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hyderabad_merged = hyderabad_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
hyderabad_merged.dropna(subset=['Cluster Labels'], inplace=True)
hyderabad_merged.head(8) # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,A. S. Rao Nagar,17.479950,78.556834,2.0,Clothing Store,Chinese Restaurant,Indian Restaurant,Department Store,Café,Diner,Women's Store,Dumpling Restaurant,Fast Food Restaurant,Farmers Market
1,A.C. Guards,17.402804,78.459487,2.0,Hyderabadi Restaurant,Indian Restaurant,Shoe Store,South Indian Restaurant,Hotel,Department Store,Middle Eastern Restaurant,Ice Cream Shop,Bakery,Café
2,Abhyudaya Nagar,17.337661,78.564716,0.0,Indian Restaurant,Hotel,Burger Joint,English Restaurant,Women's Store,Dumpling Restaurant,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,Abids,17.389478,78.477182,2.0,Hotel,Indian Restaurant,Juice Bar,Fast Food Restaurant,Electronics Store,Shoe Store,Shopping Mall,Department Store,Diner,South Indian Restaurant
4,Adikmet,17.409550,78.513094,2.0,Gym,Café,Clothing Store,Donut Shop,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
5,Aghapura,17.389178,78.465273,2.0,Afghan Restaurant,Platform,Indian Restaurant,Bakery,Women's Store,Dumpling Restaurant,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
6,Aliabad,17.345630,78.472680,2.0,Gym,Chinese Restaurant,Smoke Shop,Ice Cream Shop,Snack Place,Asian Restaurant,Women's Store,Donut Shop,Fast Food Restaurant,Farmers Market
8,Allwyn Colony,17.504362,78.414985,2.0,ATM,Bar,Mattress Store,Dumpling Restaurant,Flea Market,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant


In [150]:
hyderabad_merged['Cluster Labels'] = hyderabad_merged['Cluster Labels'].fillna(0.0)

In [151]:
hyderabad_merged['Cluster Labels'] = hyderabad_merged['Cluster Labels'].map(lambda x: int(x))

### V

In [159]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hyderabad_merged['Latitude'], hyderabad_merged['Longitude'], hyderabad_merged['Neighbourhood'], hyderabad_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster Analysis

## Cluster 1

In [164]:
hyderabad_merged.loc[hyderabad_merged['Cluster Labels'] == 0, hyderabad_merged.columns[[0, 1, 2] + list(range(4, hyderabad_merged.shape[1]))]]

,Neighbourhood,Latitude,Longitude,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
2,Abhyudaya Nagar,17.337661,78.564716,Indian Restaurant,Hotel,Burger Joint,English Restaurant,Women's Store,Dumpling Restaurant,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
10,Amberpet,17.390263,78.516481,Movie Theater,Indian Restaurant,Women's Store,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store,Dumpling Restaurant
16,Azampura,17.376193,78.490874,Indian Restaurant,Hotel,Women's Store,Donut Shop,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
20,Balkampet,17.446923,78.450451,Indian Restaurant,Hockey Arena,Train Station,Bakery,Light Rail Station,Gym,Flower Shop,Fishing Store,Fast Food Restaurant,Farmers Market
22,Bank Street,17.385717,78.480157,Indian Restaurant,Juice Bar,Hotel,Chaat Place,Department Store,South Indian Restaurant,Indie Movie Theater,Women's Store,Dumpling Restaurant,Fast Food Restaurant
27,Begum Bazaar,17.373079,78.470555,Women's Store,Indian Restaurant,Food,Harbor / Marina,Flea Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
36,Chandrayan Gutta,17.323929,78.479866,Pharmacy,Tourist Information Center,Indian Restaurant,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
48,Ferozguda,17.464618,78.461633,Indian Restaurant,Tea Room,Diner,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store,Dumpling Restaurant
52,Golnaka,17.390106,78.508360,Indian Restaurant,Food,Pizza Place,Donut Shop,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
54,Gudimalkapur,17.387920,78.437117,Indian Restaurant,Music Venue,Art Gallery,Flower Shop,Donut Shop,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant


## Cluster 2

In [165]:
hyderabad_merged.loc[hyderabad_merged['Cluster Labels'] == 1, hyderabad_merged.columns[[0, 1, 2] + list(range(4, hyderabad_merged.shape[1]))]]

,Neighbourhood,Latitude,Longitude,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
49,Film Nagar,17.414057,78.410503,Café,New American Restaurant,Greek Restaurant,Sandwich Place,Restaurant,Women's Store,Diner,Falafel Restaurant,English Restaurant,Electronics Store
66,Jamia Osmania,17.410264,78.518037,Pizza Place,Café,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store,Dumpling Restaurant
74,Karwan,17.380082,78.437475,Fried Chicken Joint,Café,Women's Store,Donut Shop,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
95,Madannapet,17.358558,78.497106,Pharmacy,Café,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store,Dumpling Restaurant
119,Mylargadda,17.422291,78.516970,Café,Women's Store,Donut Shop,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store,Dumpling Restaurant
125,Namalagundu,17.422247,78.516926,Café,Women's Store,Donut Shop,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store,Dumpling Restaurant


## Cluster 3

In [166]:
hyderabad_merged.loc[hyderabad_merged['Cluster Labels'] == 2, hyderabad_merged.columns[[0, 1, 2] + list(range(4, hyderabad_merged.shape[1]))]]

,Neighbourhood,Latitude,Longitude,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,A. S. Rao Nagar,17.479950,78.556834,Clothing Store,Chinese Restaurant,Indian Restaurant,Department Store,Café,Diner,Women's Store,Dumpling Restaurant,Fast Food Restaurant,Farmers Market
1,A.C. Guards,17.402804,78.459487,Hyderabadi Restaurant,Indian Restaurant,Shoe Store,South Indian Restaurant,Hotel,Department Store,Middle Eastern Restaurant,Ice Cream Shop,Bakery,Café
3,Abids,17.389478,78.477182,Hotel,Indian Restaurant,Juice Bar,Fast Food Restaurant,Electronics Store,Shoe Store,Shopping Mall,Department Store,Diner,South Indian Restaurant
4,Adikmet,17.409550,78.513094,Gym,Café,Clothing Store,Donut Shop,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
5,Aghapura,17.389178,78.465273,Afghan Restaurant,Platform,Indian Restaurant,Bakery,Women's Store,Dumpling Restaurant,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Shanker Mutt,17.400764,78.507340,Food Court,Ice Cream Shop,Indian Restaurant,Bakery,Dumpling Restaurant,Flea Market,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
166,Sikh Village,17.460098,78.487287,Café,Airport Terminal,Middle Eastern Restaurant,Grocery Store,Food Truck,Business Service,Indian Restaurant,Women's Store,Fast Food Restaurant,Farmers Market
168,Sindhi Colony,17.441219,78.481048,Chinese Restaurant,Ice Cream Shop,Indian Restaurant,Café,Coffee Shop,Food,Pizza Place,Smoke Shop,Sandwich Place,Bakery
169,Sitaphalmandi,17.429733,78.517426,Train Station,Grocery Store,Indian Restaurant,Bakery,Women's Store,Donut Shop,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant


## Cluster 4

In [156]:
hyderabad_merged.loc[hyderabad_merged['Cluster Labels'] == 3, hyderabad_merged.columns[[0, 1, 2] + list(range(4, hyderabad_merged.shape[1]))]]

,Latitude,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
30,17.486255,Snack Place,Pharmacy,Farmers Market,Diner,Fast Food Restaurant,Falafel Restaurant,English Restaurant,Electronics Store,Dumpling Restaurant
114,17.468531,Tea Room,Concert Hall,Donut Shop,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
136,17.415585,Comedy Club,Flea Market,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store,Dumpling Restaurant
137,17.528609,Comedy Club,Flea Market,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store,Dumpling Restaurant
152,17.357873,Train Station,Pharmacy,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store,Dumpling Restaurant
155,17.359533,Coffee Shop,Accessories Store,Dumpling Restaurant,Flea Market,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant


## Cluster 5

In [167]:
hyderabad_merged.loc[hyderabad_merged['Cluster Labels'] == 4, hyderabad_merged.columns[[0, 1, 2] + list(range(4, hyderabad_merged.shape[1]))]]

,Neighbourhood,Latitude,Longitude,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
65,Jahanuma,17.343352,78.461464,Diner,Women's Store,Dumpling Restaurant,Flea Market,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
103,Mallapur,17.440537,78.578855,Diner,Women's Store,Dumpling Restaurant,Flea Market,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
130,Nawab Saheb Kunta,17.339792,78.457836,Diner,Women's Store,Dumpling Restaurant,Flea Market,Fishing Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant,Electronics Store
